Dieses Skript dient zur Erstellung von allen Indikator-csv's, auf deutscher und niederländischer Seite. Dies sollte vollständig automatisch funktionieren.

In [2]:
import pandas as pd
import numpy as np
import re
import cbsodata # type: ignore


<h5>1. Indicatorlist einlesen</h5>
<h5>2. Indikatoren ID's Inkar abspeichern</h5>
    2.1 Für jeden Indikator von INKAR ein x in der Inkar Liste machen</br>
    2.2 Indikator von Inkar erstellen lassen</br>
    2.3 Indikator korrekte Spalten geben</br>
    2.4 Indikator richtig benennen und abspeichern</br>
<h5>3. CBS Tabellen und Spalten ID's abspeichern</h5>
    3.1 Tabelle einlesen und Spalte extrahieren</br>
    3.2 Indikator korrekte Spalten geben</br>
    3.3 Indikator richtig benennen und abspeichern</br>

Alle Daten, die vorher eingelesen werden müssen

In [7]:
# Datensatz von inkar
inkar = pd.read_csv("../../../big_data/inkar_2024.csv",sep=";",decimal=",",dtype={'Kennziffer':str,'Kennziffer_EU':str,'Name':str}, low_memory=False)
# Hilfsliste, die Informationen über deutsche Indikatoren enthält
indicator_list_de = pd.read_csv("../../out/data.csv", sep=",")
# Liste aller Indikatoren, sowohl DE als auch NL
list = pd.read_excel("../../out/indicator_list.xlsx")


<h1>1. Indicatorlist einlesen<h1>

In [8]:
list = list['source_id_de'].astype("int")

<h2>2. Indikatoren ID's Inkar abspeichern</h2>

In [14]:
indicator_id_de = list

# 2.1 Für jeden Indikator von INKAR ein x in der Inkar Liste machen

indicator_list_de['select'] = np.nan
indicator_list_de.loc[indicator_list_de['source_id_de'].isin(indicator_id_de), 'select'] = 'x'

x_count = indicator_list_de['select'].value_counts().get('x', 0)
print(f"\nAnzahl der 'x' in der Spalte 'select': {x_count}")


Anzahl der 'x' in der Spalte 'select': 76


In [90]:
# 2.2 Indikator von Inkar erstellen lassen

indicators = pd.DataFrame(columns=['category_de', 'name_de', 'source_de', 'source_id_de'])

# Es werden von jeden Indikator, der ausgewählt wurde (x), die wichtigen Parameter abgefragt und abgespeichert
for index in indicator_list_de.index:
    if indicator_list_de['select'][index] == "x":
        category_value = indicator_list_de['category_de'][index]
        indicator_name = indicator_list_de['name_de'][index]
        source_value = indicator_list_de['source_de'][index]
        source_id_value = int(indicator_list_de['source_id_de'][index])
        indicators.loc[len(indicators)] = {'category_de': category_value, 'name_de': indicator_name, 'source_de': source_value, 'source_id_de':source_id_value}

indicators.reset_index(drop=True, inplace=True)

# Filter festlegen: Raum- und Zeitbezug
raumbezug_selection = ["Kreise","Gemeinden", "Bund", "Bundesländer", "Gemeindeverbände"]
inkar = inkar[inkar['Raumbezug'].isin(raumbezug_selection)]

zeitbezug_selection = [2022]
inkar = inkar[inkar['Zeitbezug'].isin(zeitbezug_selection)]


filtered_inkar = inkar[(inkar['Bereich'].isin(indicators['category_de'])) & (inkar['ID'].isin(indicators['source_id_de'])) & (inkar['Indikator'].isin(indicators['name_de']))]
filtered_inkar.reset_index(drop=True, inplace=True)

indicators_csv = filtered_inkar['Indikator'].unique()

# 2.3 Indikator korrekte Spalten geben
wanted_columns = ["Kennziffer", "Wert", "Zeitbezug"]

# Abschließende Schritt, für jeden Indikator werden die csv's erstellt
for indikator in indicators_csv:
    
    currentData = filtered_inkar[filtered_inkar['Indikator'] == indikator]
    currentData = currentData[wanted_columns]

    # 2.4 Indikator richtig benennen und abspeichern
    currentData = currentData.rename(columns={'Kennziffer': 'ID', 'Wert': 'Value', 'Zeitbezug' : 'Year'})

    indicator_ids = filtered_inkar[filtered_inkar['Indikator'] == indikator]['ID'].unique()
    if len(indicator_ids) > 0:
        indicator_id = indicator_ids[0]
    
    fileName = f"C:/Users/juliu/VSCODE/VSCODE/inkar-indicators/src/out/data/Inkar/{indicator_id}.csv"
    currentData.to_csv(fileName, index=False)

<h1>3. CBS Tabellen und Spalten ID's abspeichern</h1>

In [19]:
#3. CBS Tabellen und Spalten ID's abspeichern

list = pd.read_excel("../../out/indicator_list.xlsx")

temp_list = list[['source_id_nl', 'source_id_value_nl']]

temp_list['source_id_nl'] = temp_list['source_id_nl'].astype(str)
temp_list['source_id_value_nl'] = temp_list['source_id_value_nl'].astype(str) 

# Funktion zur Auswhl der Zahlen nach dem letzten Unterstrich
def extract_last_numbers(value):
    # Aufsplitten des Strings an "&" und dann nur die Zahl nach dem letzten Unterstrich nehmen
    parts = value.split(" & ")
    numbers = [re.search(r'_(\d+)$', part).group(1) for part in parts if re.search(r'_(\d+)$', part)]
    return "_".join(numbers)

# Neue Spalte erstellen, indem die Zahlen und die 'source_id_nl' verbunden werden
temp_list['combined'] = temp_list['source_id_nl'].astype(str) + '_' + temp_list['source_id_value_nl'].apply(extract_last_numbers)

temp_list


C:\Users\juliu\AppData\Local\Temp\ipykernel_19184\870224416.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_list['source_id_nl'] = temp_list['source_id_nl'].astype(str)
C:\Users\juliu\AppData\Local\Temp\ipykernel_19184\870224416.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_list['source_id_value_nl'] = temp_list['source_id_value_nl'].astype(str)
C:\Users\juliu\AppData\Local\Temp\ipykernel_19184\870224416.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,source_id_nl,source_id_value_nl,combined
0,85826NED,Werkloosheidspercentage_13,85826NED_13
1,85826NED,NettoArbeidsparticipatie_16,85826NED_16
2,85796NED,Werknemer_4,85796NED_4
3,85796NED,Werknemer_4,85796NED_4
4,84780NED,VoortijdigSchoolverlaters_2,84780NED_2
...,...,...,...
73,85701NED,A025280,85701NED_
74,85701NED,T001345,85701NED_
75,85701NED,A041867,85701NED_
76,80305ned,AfstandTotBibliotheek_103,80305ned_103


In [3]:
#3.1 Tabelle einlesen und Spalte extrahieren
#3.2 Indikator korrekte Spalten geben
#3.3 Indikator richtig benennen und abspeichern

In [4]:
list = pd.read_excel("out/indicator_list_2.xlsx")
list

,category_de,name_de,source_de,source_id_de,category_nl,name_nl,source_nl,source_id_nl,source_id_value_nl,source_URL_nl_regional,Similarity indicator,Remarks
0,Arbeitslosigkeit,Arbeitslosenquote,Inkar,14574,Arbeidsdeelname,Arbeidsdeelname; regionale indeling 2023,CBS Open data Statline,85826NED,Werkloosheidspercentage_13,https://opendata.cbs.nl/statline/portal.html?_...,1.0,NaN
1,Beschäftigung und Erwerbstätigkeit,Beschäftigtenquote,Inkar,17508,Arbeidsdeelname,Arbeidsdeelname; regionale indeling 2023,CBS Open data Statline,85826NED,NettoArbeidsparticipatie_16,https://opendata.cbs.nl/statline/portal.html?_...,1.0,NaN
2,Beschäftigung und Erwerbstätigkeit,Beschäftigte am WO mit akademischen Abschluss,Inkar,18167,Arbeidsdeelname,Arbeidsdeelname; gemeenten,CBS Open data Statline,85796NED,Werknemer_4,https://opendata.cbs.nl/statline/portal.html?_...,1.0,NaN
3,Beschäftigung und Erwerbstätigkeit,Beschäftigte am WO ohne Berufsabschluss,Inkar,18487,Arbeidsdeelname,Arbeidsdeelname; gemeenten,CBS Open data Statline,85796NED,Werknemer_4,https://opendata.cbs.nl/statline/portal.html?_...,1.0,NaN
4,Bildung,Schulabgänger ohne Abschluss,Inkar,14166,Voortijdig schoolverlaters (VSV),"Vsv; geslacht, woonregio",CBS Open data Statline,84780NED,VoortijdigSchoolverlaters_2,https://opendata.cbs.nl/statline/portal.html?_...,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
73,Zentrale Orte Monitoring,Schüler an Grundschulen,Inkar,156060,"Leerlingen en studenten; onderwijssoort, woonr...",Primary education,CBS Open data Statline,85701NED,A025280,https://opendata.cbs.nl/statline/portal.html?_...,1.0,"I am not sure if in Germany this also entail ""..."
74,Zentrale Orte Monitoring,Weiterführende Schulen,Inkar,156110,"Leerlingen en studenten; onderwijssoort, woonr...",Secondary schools,CBS Open data Statline,85701NED,T001345,https://opendata.cbs.nl/statline/portal.html?_...,NaN,NaN
75,Zentrale Orte Monitoring,Berufsbildende Schulen,Inkar,156170,"Leerlingen en studenten; onderwijssoort, woonr...",Vocational schools = beroepsonderwijs,CBS Open data Statline,85701NED,A041867,https://opendata.cbs.nl/statline/portal.html?_...,NaN,NaN
76,Zentrale Orte Monitoring,Bibliotheken,Inkar,156200,Nabijheid voorzieningen; regio,NaN,CBS Open data Statline,80305ned,AfstandTotBibliotheek_103,https://opendata.cbs.nl/statline/portal.html?_...,3.0,"Germany: number of libraries, Netherland: dist..."


In [19]:
mapping = ["0", "0", "0", "0", "1", "0", "0", "0", "0", "0", "0", "1", "0","0", "0", "0", "0", "0", "0", "0", "0","0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0","0", "0", "0", "0", "0", "0", "0", "0","0", "0", "0", "0", "0", "0", "0", "0","0", "0", "0", "0", "0", "0", "0", "0","0", "0", "0", "0", "0", "0", "0", "0","0", "0", "0", "0", "0", "0", "0", "0", "0"]



list['Mapping'] = mapping

def select_regioS (table, column, filter):

    data = pd.DataFrame(
        cbsodata.get_data(table_id, 
                          filters="Perioden eq '2022JJ00' and Persoonskenmerken eq 'T009002'",
                          select=['RegioS', table_column_id, 'Perioden'])
                          )
    
    #wanted_columns = ["RegioS",table_column_id, "Perioden"]
    #currentData = data[wanted_columns]
    currentData = data.rename(columns={'RegioS': 'ID', table_column_id: 'Value', 'Perioden' : 'Year'})

    print(f"Table:  {table_id}   Column: {table_column_id}")
    print(currentData.head())

    return currentData

def select_RegioMerken (table, column, filter):

    data = pd.DataFrame(
        cbsodata.get_data(table_id, 
                          filters="Perioden eq '2022JJ00' and Persoonskenmerken eq 'T009002'",
                          select=['Regiokenmerken', table_column_id, 'Perioden'])
                          )
    
    #wanted_columns = ["Regiokenmerken",table_column_id, "Perioden"]
    #currentData = data[wanted_columns]
    currentData = data.rename(columns={'Regiokenmerken': 'ID', table_column_id: 'Value', 'Perioden' : 'Year'})

    print(f"Table:  {table_id}   Column: {table_column_id}")
    print(currentData.head())

    return currentData

for index in list.index:
    table_id = list['source_id_nl'][index]
    table_column_id = str(list['source_id_value_nl'][index])

    if '&' in table_column_id:
        ids_split = table_column_id.split('&')
        ids_split = [id.strip() for id in ids_split]
        table_column_id = ', '.join(ids_split)


    if (list['Mapping'][index] == "0"):
        select_regioS(table=table_id, column=table_column_id, filter="")
    elif(list['Mapping'][index] == "1"):
        select_RegioMerken(table=table_id, column=table_column_id, filter="")

Table:  85826NED   Column: Werkloosheidspercentage_13
                     ID  Value  Year
0             Nederland    3.5  2022
1  Noord-Nederland (LD)    3.6  2022
2   Oost-Nederland (LD)    3.3  2022
3   West-Nederland (LD)    3.7  2022
4   Zuid-Nederland (LD)    3.3  2022
Table:  85826NED   Column: NettoArbeidsparticipatie_16
                     ID  Value  Year
0             Nederland   72.2  2022
1  Noord-Nederland (LD)   70.9  2022
2   Oost-Nederland (LD)   73.1  2022
3   West-Nederland (LD)   72.4  2022
4   Zuid-Nederland (LD)   71.4  2022
Table:  85796NED   Column: Werknemer_4
                                  ID  Value  Year
0                          Nederland   83.0  2022
1               Noord-Nederland (LD)   84.0  2022
2                Oost-Nederland (LD)   84.0  2022
3                West-Nederland (LD)   83.0  2022
4  West-Nederland, exclusief G4 (LD)    NaN  2022
Table:  85796NED   Column: Werknemer_4
                                  ID  Value  Year
0                  

HTTPError: Downloading table '85701NED' failed. 500 Server Error: Could not find a property named 'A025294' on type 'Cbs.OData.TData'. for url: https://opendata.cbs.nl/ODataFeed/odata/85701NED/TypedDataSet?%24format=json&%24select=Regiokenmerken%2CA025294%2C+A025297%2CPerioden&%24filter=Perioden+eq+%272022JJ00%27+and+Persoonskenmerken+eq+%27T009002%27

In [17]:
da = pd.DataFrame(cbsodata.get_data("85701NED",
                  #filters=""
                  ))
da


,ID,Geslacht,Onderwijssoort,Regiokenmerken,Perioden,LeerlingenStudenten_1
0,0,Totaal mannen en vrouwen,Totaal (speciaal) basisonderwijs,Nederland,2011/'12,1559270.0
1,1,Totaal mannen en vrouwen,Totaal (speciaal) basisonderwijs,Nederland,2012/'13,1537930.0
2,2,Totaal mannen en vrouwen,Totaal (speciaal) basisonderwijs,Nederland,2013/'14,1514860.0
3,3,Totaal mannen en vrouwen,Totaal (speciaal) basisonderwijs,Nederland,2014/'15,1494400.0
4,4,Totaal mannen en vrouwen,Totaal (speciaal) basisonderwijs,Nederland,2015/'16,1477850.0
...,...,...,...,...,...,...
477355,477355,Vrouwen,Wetenschappelijk onderwijs,Niet in te delen,2019/'20,8890.0
477356,477356,Vrouwen,Wetenschappelijk onderwijs,Niet in te delen,2020/'21,11930.0
477357,477357,Vrouwen,Wetenschappelijk onderwijs,Niet in te delen,2021/'22,12630.0
477358,477358,Vrouwen,Wetenschappelijk onderwijs,Niet in te delen,2022/'23,10910.0


In [150]:
d = pd.DataFrame(cbsodata.get_meta("85711NED", "TypedDataSet"))
d

KeyboardInterrupt: 

In [41]:
d['Onderwijssoort'].unique()

array(['A025279', 'A025280', 'A025281', 'A025282', 'A049120', 'A049121',
       'T001345', 'A041919', 'A041920', 'A041921', 'A041922', 'A041923',
       'A025289', 'A025300', 'A043035', 'A043032', 'A043029', 'A041867',
       'A042650', 'A042656', 'A042662', 'A042668', 'A025294', 'A025297'],
      dtype=object)